# YOLO License Plate Recognition - Từ Zero đến Hero

Notebook này sẽ hướng dẫn bạn:
1. Hiểu YOLO hoạt động như thế nào
2. Chạy YOLO có sẵn để detect đối tượng
3. Fine-tune YOLO cho biển số xe Việt Nam
4. Tối ưu cho real-time processing

## Bước 1: Import thư viện và kiểm tra cài đặt

Trước tiên, hãy cài đặt các thư viện cần thiết:
```bash
pip install ultralytics torch torchvision pillow matplotlib
```

In [2]:
# Import các thư viện cần thiết
import cv2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

In [4]:
# Import YOLO từ Ultralytics
try:
    from ultralytics import YOLO
    print('✅ Ultralytics YOLO đã được cài đặt thành công!')
except ImportError:
    print('❌ Cần cài đặt: pip install ultralytics')

# Kiểm tra PyTorch
try:
    import torch
    print(f'✅ PyTorch version: {torch.__version__}')
    print(f'✅ CUDA available: {torch.cuda.is_available()}')
    if torch.cuda.is_available():
        print(f'✅ GPU: {torch.cuda.get_device_name(0)}')
    else:
        print('⚠️  GPU không có sẵn, sẽ dùng CPU (chậm hơn)')
except ImportError:
    print('❌ Cần cài đặt: pip install torch torchvision')

✅ Ultralytics YOLO đã được cài đặt thành công!
✅ PyTorch version: 2.8.0+cu128
✅ CUDA available: True
✅ GPU: NVIDIA GeForce RTX 3050 Laptop GPU


## Bước 2: Tải và chạy thử YOLO có sẵn

Chúng ta sẽ bắt đầu với model YOLOv8 đã được train trên COCO dataset (80 classes bao gồm xe, người, v.v.)

**Giải thích:**
- `yolov8n.pt`: nano version (nhỏ nhất, nhanh nhất)
- `yolov8s.pt`: small version
- `yolov8m.pt`: medium version
- `yolov8l.pt`: large version
- `yolov8x.pt`: extra large (chính xác nhất, chậm nhất)

In [ ]:
# Tải model YOLO có sẵn (sẽ tự động download lần đầu)
print("📥 Đang tải YOLOv8 nano model...")
model = YOLO("yolov8n.pt")  # Dùng nano version cho tốc độ

print("✅ Model đã sẵn sàng!")
print(f"📊 Model có thể detect {len(model.names)} classes:")

# In ra các class model có thể detect
for i, name in model.names.items():
    print(f"{i:2d}: {name}")

# Tìm class liên quan đến vehicle
vehicle_classes = [
    (i, name)
    for i, name in model.names.items()
    if any(keyword in name.lower() for keyword in ["car", "truck", "bus", "motorcycle"])
]
print(f"\n🚗 Vehicle classes: {vehicle_classes}")

## Bước 3: Test Model Performance

Bây giờ ta sẽ test model với:
1. **Ảnh demo** có biển số xe Việt Nam
2. **Đo tốc độ inference** (FPS) để đảm bảo đạt yêu cầu 4-10 FPS
3. **Visualize kết quả** để hiểu model detect được gì
4. **Phân tích gap** giữa kết quả hiện tại vs mục tiêu

**Lưu ý:** Model hiện tại chưa được train cho biển số, nên kết quả sẽ không hoàn hảo. Đây là baseline để so sánh.

In [ ]:
# Load ảnh thật từ thư mục images/ để test
def list_images(folder='images'):
    exts = ('.jpg', '.jpeg', '.png', '.bmp')
    folder_path = Path(folder)
    if not folder_path.exists():
        folder_path.mkdir(parents=True, exist_ok=True)
    files = [p for p in folder_path.iterdir() if p.suffix.lower() in exts]
    return sorted(files)

image_files = list_images('images')

if not image_files:
    print('⚠️ Không tìm thấy ảnh trong thư mục images/')
    print('👉 Hãy thêm một vài ảnh biển số thật vào thư mục: images/')
    print('Ví dụ tên file: car1.jpg, plate01.png ...')
else:
    print(f'📁 Tìm thấy {len(image_files)} ảnh:')
    for idx, fp in enumerate(image_files):
        print(f'  [{idx}] {fp.name}')
    
    # Chọn ảnh theo index
    idx = 0  # Bạn có thể đổi số này để chọn ảnh khác
    selected_image_path = image_files[idx]
    print(f'\n✅ Đang dùng ảnh: {selected_image_path.name}')
    
    # Load ảnh
    real_img = cv2.imread(str(selected_image_path))
    if real_img is None:
        print('❌ Không load được ảnh (có thể file hỏng).')
    else:
        # Hiển thị ảnh
        plt.figure(figsize=(10, 6))
        plt.imshow(cv2.cvtColor(real_img, cv2.COLOR_BGR2RGB))
        plt.title(f'🖼️ Ảnh thật: {selected_image_path.name}', fontsize=14)
        plt.axis('off')
        plt.show()
        
        # Gán ảnh này làm ảnh test cho YOLO
        demo_img = real_img.copy()
        print('🔄 Đã thay ảnh demo bằng ảnh thật để inference!')

In [10]:
# Chạy YOLO inference trên ảnh demo
print('🔍 Running YOLO detection on demo image...')

# Inference (chạy model)
results = model(demo_img, verbose=False)

# Lấy kết quả đầu tiên
result = results[0]
boxes = result.boxes

print(f'\n📊 **DETECTION RESULTS:**')
print(f'🔢 Number of detections: {len(boxes) if boxes is not None else 0}')

# Phân tích từng detection
if boxes is not None and len(boxes) > 0:
    print('\n📋 **DETECTED OBJECTS:**')
    for i, box in enumerate(boxes):
        # Extract thông tin từ box
        xyxy = box.xyxy[0].cpu().numpy()  # [x1, y1, x2, y2]
        conf = float(box.conf[0].cpu().numpy())  # confidence score
        cls = int(box.cls[0].cpu().numpy())      # class id
        class_name = model.names[cls]
        
        x1, y1, x2, y2 = xyxy
        width = x2 - x1
        height = y2 - y1
        
        print(f'   Detection {i+1}:')
        print(f'     🏷️  Class: {class_name} (ID: {cls})')
        print(f'     📊 Confidence: {conf:.3f} ({conf*100:.1f}%)')
        print(f'     📐 Bounding box: ({x1:.0f}, {y1:.0f}) → ({x2:.0f}, {y2:.0f})')
        print(f'     📏 Size: {width:.0f} × {height:.0f} pixels')
        
        # Check if detected the license plate area
        if 230 <= (x1+x2)/2 <= 390 and 350 <= (y1+y2)/2 <= 410:
            print(f'     ✅ Có thể đây là vùng biển số!')
        print()

    # Visualize kết quả
    annotated_img = result.plot(line_width=3, font_size=16)
    
    # Display side-by-side comparison
    plt.figure(figsize=(16, 8))
    
    plt.subplot(1, 2, 1)
    plt.imshow(cv2.cvtColor(demo_img, cv2.COLOR_BGR2RGB))
    plt.title('🖼️ Original Demo Image', fontsize=14, fontweight='bold')
    plt.axis('off')
    
    plt.subplot(1, 2, 2)
    plt.imshow(cv2.cvtColor(annotated_img, cv2.COLOR_BGR2RGB))
    plt.title('🎯 YOLO Detection Results', fontsize=14, fontweight='bold')
    plt.axis('off')
    
    plt.tight_layout()
    plt.show()

else:
    print('❌ No objects detected!')
    print('💭 Possible reasons:')
    print('   - Model confidence threshold too high')
    print('   - Demo image too simple/synthetic')
    print('   - Need to adjust inference parameters')

print('\n🎯 **NEXT STEP:** Đo tốc độ inference (FPS testing)')

🔍 Running YOLO detection on demo image...

📊 **DETECTION RESULTS:**
🔢 Number of detections: 0
❌ No objects detected!
💭 Possible reasons:
   - Model confidence threshold too high
   - Demo image too simple/synthetic
   - Need to adjust inference parameters

🎯 **NEXT STEP:** Đo tốc độ inference (FPS testing)


In [ ]:
# Benchmark tốc độ inference để đảm bảo đạt 4-10 FPS
import time

print('⏱️ Testing inference speed...')

# Tạo ảnh dummy để test (640x640 standard YOLO input)
dummy_image = np.random.randint(0, 255, (640, 640, 3), dtype=np.uint8)

# Warm-up (model cần một vài lần chạy đầu để tối ưu)
print('🔥 Warming up model...')
for _ in range(3):
    _ = model(dummy_image, verbose=False)

# Benchmark thực tế
print('📊 Running benchmark (10 iterations)...')
times = []
for i in range(10):
    start_time = time.time()
    results = model(dummy_image, verbose=False)
    end_time = time.time()
    
    inference_time = end_time - start_time
    fps = 1 / inference_time
    times.append(inference_time)
    
    print(f'  Frame {i+1}: {inference_time*1000:.1f}ms → {fps:.1f} FPS')

# Thống kê
avg_time = np.mean(times)
avg_fps = 1 / avg_time
min_time = np.min(times)
max_fps = 1 / min_time

print(f'\n📈 **PERFORMANCE SUMMARY:**')
print(f'⏱️  Average inference time: {avg_time*1000:.1f}ms')
print(f'🚀 Average FPS: {avg_fps:.1f}')
print(f'🏆 Best FPS: {max_fps:.1f}')
print(f'🎯 Target range: 4-10 FPS')

# Đánh giá kết quả
if avg_fps < 4:
    print('❌ **PERFORMANCE: TOO SLOW**')
    print('💡 Recommendations:')
    print('   - Giảm input size: 640→480 hoặc 320')
    print('   - Sử dụng GPU nếu có')
    print('   - Consider model quantization')
elif avg_fps > 10:
    print('✅ **PERFORMANCE: EXCELLENT!**')
    print('🎉 Options:')
    print('   - Có thể tăng input size → better accuracy')
    print('   - Hoặc dùng model lớn hơn (yolov8s)')
    print('   - Hardware của bạn rất tốt!')
else:
    print('✅ **PERFORMANCE: PERFECT!**')
    print('🎯 Đạt chính xác target range 4-10 FPS')
    print('⚡ Sẵn sàng cho real-time processing!')

print(f'\n💻 **SYSTEM INFO:**')
print(f'🔧 Model: YOLOv8n')
print(f'🖼️  Input size: 640x640')
print(f'🎮 Device: {"GPU" if torch.cuda.is_available() else "CPU"}')